# The Best Gala Agent

- We decided to host the most extravagant and opulent party of the century. This means lavish feasts, enchanting dancers, renowned DJs, exquisite drinks, a breathtaking fireworks display, and much more.
- Our party planner agent will manage everything by himself. To do so, he needs to have access to all of the information about the party, including the menu, the guests, the schedule, weather forecasts, and much more!
- He also needs to be able to answer any questions about the party during the party
- Requirements:
    - Be aware of topics, like politics and religion, that are to be avoided at a gala
    - A good host should be aware of guests’ backgrounds
    - Some general knowledge about the weather to ensure we can continuously find a real-time update to ensure perfect timing to launch the fireworks

### Why RAG for a Gala?

During the party, our agent will need to recall specific details about each person at a moment’s notice. A traditional LLM might struggle with this task because:

1. The guest list is specific to your event and not in the model’s training data
2. Guest information may change or be updated frequently
3. Alfred needs to retrieve precise details like email addresses

In [1]:
# Step 1: Load & Prepare the Data

import datasets
from langchain.docstore.document import Document

# Load the dataset
guest_dataset = datasets.load_dataset("agents-course/unit3-invitees", split="train")

# Convert dataset entries into Document objects
docs = [
    Document(
        page_content="\n".join([
            f"Name: {guest['name']}",
            f"Relation: {guest['relation']}",
            f"Description: {guest['description']}",
            f"Email: {guest['email']}"
        ]),
        metadata={"name": guest["name"]}
    )
    for guest in guest_dataset
]


In [2]:
docs

[Document(metadata={'name': 'Ada Lovelace'}, page_content="Name: Ada Lovelace\nRelation: best friend\nDescription: Lady Ada Lovelace is my best friend. She is an esteemed mathematician and friend. She is renowned for her pioneering work in mathematics and computing, often celebrated as the first computer programmer due to her work on Charles Babbage's Analytical Engine.\nEmail: ada.lovelace@example.com"),
 Document(metadata={'name': 'Dr. Nikola Tesla'}, page_content="Name: Dr. Nikola Tesla\nRelation: old friend from university days\nDescription: Dr. Nikola Tesla is an old friend from your university days. He's recently patented a new wireless energy transmission system and would be delighted to discuss it with you. Just remember he's passionate about pigeons, so that might make for good small talk.\nEmail: nikola.tesla@gmail.com"),
 Document(metadata={'name': 'Marie Curie'}, page_content='Name: Marie Curie\nRelation: no relation\nDescription: Marie Curie was a groundbreaking physicist 

In [3]:
# Step 2: Create a Retriever Tool
# We will use the BM25Retriever from the langchain_community.retrievers module to create a retriever tool.


from langchain_community.retrievers import BM25Retriever
from langchain.tools import Tool

bm25_retriever = BM25Retriever.from_documents(docs)

def extract_text(query: str) -> str:
    """Retrieves detailed information about gala guests based on their name or relation."""
    results = bm25_retriever.invoke(query)
    if results:
        return "\n\n".join([doc.page_content for doc in results[:3]])
    else:
        return "No matching guest information found."

guest_info_tool = Tool(
    name="guest_info_retriever",
    func=extract_text,
    description="Retrieves detailed information about gala guests based on their name or relation."
)


### This tool: 
- The name & description help the agent understand when and how to use this tool
- The type decorator define what parameters the tool expects (in this case a search query)
- BM25 doesn't require embeddings

In [4]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
from langchain_core.messages import AnyMessage, HumanMessage, AIMessage
from langgraph.prebuilt import ToolNode
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition
from langchain_ollama import ChatOllama

# Generate the chat interface, including the tools
llm = "qwen2.5-coder:7b"

chat = ChatOllama(model=llm, verbose=True)
tools = [guest_info_tool]
chat_with_tools = chat.bind_tools(tools)


In [5]:
# Step 3: Integrate the Tool with the Agent

# Generate the AgentState and Agent graph
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

def assistant(state: AgentState):
    return {""
        "messages": [chat_with_tools.invoke(state["messages"])],
    }

## The graph
builder = StateGraph(AgentState)

# Define nodes: these do the work
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges: these determine how the control flow moves
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    # If the latest message requires a tool, route to tools
    # Otherwise, provide a direct response
    tools_condition,
)
builder.add_edge("tools", "assistant")
agent = builder.compile()

messages = [HumanMessage(content="Tell me about our guest named 'Ada Lovelace'. Make sure to not include information about other guests.")]
response = agent.invoke({"messages": messages})

print("🎩 Party Planner's Response:")
print(response['messages'][-1].content)


🎩 Party Planner's Response:
Ada Lovelace was an esteemed mathematician and friend who is renowned for her pioneering work in mathematics and computing, often celebrated as the first computer programmer due to her work on Charles Babbage's Analytical Engine. Her email address is ada.lovelace@example.com.


### Building and Integrating Tools for Agent

- Web Search Tool
- Creating a Custom Tool for Weather Information to Schedule the Fireworks
- Creating a Hub Stats Tool for Influential AI Builders

In [8]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()
results = search_tool.invoke("Who's the current President of France?")
print(results)

Emmanuel Macron (born December 21, 1977, Amiens, France) is a French banker and politician who was elected president of France in 2017. Macron was the first person in the history of the Fifth Republic to win the presidency without the backing of either the Socialists or the Gaullists, and he was France's youngest head of state since Napoleon I.He was reelected in 2022, becoming the first ... The current President of France is Emmanuel Macron, who has held office since being elected in the 2017 French Presidential Election. The Prime Minister of France is the leader of government and holds the power to manage the numerous public agencies based around the nation. The French Executive Cabinet is formally comprised of members of ... PARIS (AP) — French President Emmanuel Macron vowed Thursday to stay in office until the end of his term, due in 2027, and announced that he will name a new prime minister within days following ... Emmanuel Macron is the current President of France, serving sin

In [9]:
from langchain.tools import Tool
import random

def get_weather_info(location: str) -> str:
    """Fetches dummy weather information for a given location."""
    # Dummy weather data
    weather_conditions = [
        {"condition": "Rainy", "temp_c": 15},
        {"condition": "Clear", "temp_c": 25},
        {"condition": "Windy", "temp_c": 20}
    ]
    # Randomly select a weather condition
    data = random.choice(weather_conditions)
    return f"Weather in {location}: {data['condition']}, {data['temp_c']}°C"

# Initialize the tool
weather_info_tool = Tool(
    name="get_weather_info",
    func=get_weather_info,
    description="Fetches dummy weather information for a given location."
)


- Since there are influential AI builders, our party planner agent should impress them by discussing their most popular models and pieces of works.

In [10]:
from huggingface_hub import list_models

def get_hub_stats(author: str) -> str:
    """Fetches the most downloaded model from a specific author on the Hugging Face Hub."""
    try:
        # List models from the specified author, sorted by downloads
        models = list(list_models(author=author, sort="downloads", direction=-1, limit=1))

        if models:
            model = models[0]
            return f"The most downloaded model by {author} is {model.id} with {model.downloads:,} downloads."
        else:
            return f"No models found for author {author}."
    except Exception as e:
        return f"Error fetching models for {author}: {str(e)}"

# Initialize the tool
hub_stats_tool = Tool(
    name="get_hub_stats",
    func=get_hub_stats,
    description="Fetches the most downloaded model from a specific author on the Hugging Face Hub."
)

# Example usage
print(hub_stats_tool("facebook")) # Example: Get the most downloaded model by Facebook

The most downloaded model by facebook is facebook/esmfold_v1 with 13,602,902 downloads.


C:\Users\ericw\AppData\Local\Temp\ipykernel_25428\4027241918.py:25: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(hub_stats_tool("facebook")) # Example: Get the most downloaded model by Facebook


### Now let's integrate these tools into our party planner agent

In [12]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
from langchain_core.messages import AnyMessage, HumanMessage, AIMessage
from langgraph.prebuilt import ToolNode
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

# Generate the chat interface, including the tools
llm = "qwen2.5-coder:7b"

chat = ChatOllama(model=llm, verbose=True)

tools = [search_tool, weather_info_tool, hub_stats_tool]
chat_with_tools = chat.bind_tools(tools)

# Generate the AgentState and Agent graph
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

def assistant(state: AgentState):
    return {
        "messages": [chat_with_tools.invoke(state["messages"])],
    }

## The graph
builder = StateGraph(AgentState)

# Define nodes: these do the work
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges: these determine how the control flow moves
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    # If the latest message requires a tool, route to tools
    # Otherwise, provide a direct response
    tools_condition,
)
builder.add_edge("tools", "assistant")
alfred = builder.compile()

messages = [HumanMessage(content="Who is Google and what's their most popular model?")]
response = alfred.invoke({"messages": messages})

print("🎩 Party Planner's Response:")
print(response['messages'][-1].content)

🎩 Party Planner's Response:
Based on the provided information, here's a summary:

Google is a multinational technology company known for its search engine, which has been a cornerstone of its business model. The company also develops and releases AI models that are designed to enhance various functionalities.

### Most Popular Model:
Google has recently announced an "enhanced" reasoning mode for its flagship Gemini 2.5 Pro model called Deep Think. Additionally, the company released Gemini 2.0, which is described as their most capable artificial intelligence model suite yet.

These advancements suggest that Google continues to push the boundaries of AI capabilities and aims to provide more efficient and accurate services to its users through its innovative models.

### Key Points:
1. **Initial Business Model**: Google initially built a powerful search engine using advanced algorithms.
2. **AI Models**: The company is known for developing and releasing AI models like Gemini 2.5 Pro (with